***
# **Tarea 1 - Análisis de Datos**
***

## **Web scraping, Manejo de datos y Probabilidades**

- **Profesores:**  Ricardo Flores (rflores@inf.udec.cl).
- **Ayudante:** Felipe Burgos Rubilar (felipeiburgos@udec.cl).

- Fecha de entrega: **11/09/25**

- **¿Cómo?:** La tarea debe ser subida a CANVAS en un archivo con el formato **NombreApellido_T1.zip**. Este debe contener el código solicitado, junto con el conjunto de datos utilizado.

***
#### **Normas del curso:**
Antes de comenzar lea detenidamente las normas a aplicar en la evaluación de esta tarea.

- El horario límite para entregar la tarea es a las **23:59** de la fecha señalada en el encabezado. Entregas posteriores a la fecha y hora indicada utilizan uno de sus días permitidos para atraso, en caso de no tener días de atraso disponibles se asigna **nota mínima**. Es por esto que se le recomienda subir la tarea con anticipación.
- Cada respuesta debe ser explicada claramente sin dejar espacio a dobles interpretaciones. **El código de la tarea debe estar comentado.**
- Está bien discutir las respuestas con compañeros/as. Está bien buscar la respuesta en internet. Está bien asistir a clases donde comentaremos como resolver la tarea. **No está bien** entregar código escrito por otras personas ya sea compañeros, código obtenido por internet o incluso el código que el profesor programa en clases.
- **Todo código entregado debe ser escrito completamente por usted**. De lo contrario, su tarea será penalizada con la nota mínima.

***

## **$\bullet$ Parte 1: Web scraping (30 ptos)**
El ayudante quiere comenzar su propio canal de criticas y reseñas sobre series de televisión y películas quieren dar una dura pelea a las grandes influencers como CoffeTV, VSXProject, ZEPfilms, etc. Para esto el ayudante ha consumido cuantas series ha podido. Sin embargo, a el también le gustaría tener información actualizada sobre las series disponibles, en todo momento. El ayudante no han tenido tiempo para realizar esto, por lo cual le solicitan ayuda al curso de “Análisis de Datos” del Departamento de Informática de la UdeC para que les ayuden con el script y el análisis.

En base a esto se le solicita a usted que genere un código que permita extraer la información sobre series de television de la página sensacine.com/series-tv filtrados por género, ya que recopila información de varios géneros y paises y no tiene restricciones de extracción de datos mediante técnicas de scraping (más o menos).

### Los datos que les interesan al ayudante son los siguientes:

**Información que se puede extraer desde la página general:**

- Filtrado según género.

**Información que se puede extraer desde la página por género:**

*Identificacion*
- Nombre de la Serie.

*Puntuación*
- Puntuación de los medios.
- Puntuacion de los usuarios.

*Emision:*
- Año de lanzamiento.
- Año de termino.

**Información que deberán extraer dentro de la página de cada serie:**
- Género y subjeneros.
- Cantidad de temporadas.
- Cantidad de episodios.
- Donde se puede ver (servicio de streaming).

Como habrán aprendido en clases, en el proceso de recolección de datos no siempre encontrarán toda la información en cada uno de los productos, por lo cual deberán decidir en cómo trabajar con estos tipos de datos faltantes. Para esto tendrán que explicar cada decisión tomada para la representación de estos. Para efectos de revisión, todos los datos obtenidos deberán ser guardados en un archivo ".pkl" (pickle), ya que la página es actualizada constantemente.

**Resumen:** En esta primera parte se espera que extraigan la información **(15 pts)**, la almacenen en un DataFrame **(2 pts)**, que limpien los datos **(3 pts)**, generen las variables de interes correspondientes en un formato correcto **(7 pts)**, que expliquen las decisiones tomadas **(3 pts)** y guarden los datos en un archivo “.pkl”.

### **$\bullet$ Librerías**

In [1]:
# Instalación de librerias para el scrapping -----------------------------------
!pip install requests
!pip install bs4
#-------------------------------------------------------------------------------

In [ ]:
# Puede utilizar más librerias si lo considera necesario------------------------

# Web Scraping------------------------------------------------------------------
import requests
import bs4

# Data handling-----------------------------------------------------------------
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re

# Computation-------------------------------------------------------------------
import time

# Varios -----------------------------------------------------------------------
import pickle
import random
import os

#-------------------------------------------------------------------------------

In [ ]:
# Conect drive -----------------------------------------------------------------
from google.colab import drive

drive.mount('/content/drive')
#-------------------------------------------------------------------------------

In [ ]:
#-------------------------------------------------------------------------------
base_url = "https://www.sensacine.com/series-tv/"
#-------------------------------------------------------------------------------

## **$\bullet$ Parte 2: Manejo de datos (30 ptos)**

En primera al ayudante le gustaría un análisis de filtrado de las series a nivel de generos y año de lanzamiento. Para esto le piden a usted que responda las siguientes preguntas:

### Pregunta 2.1 (4 pts):

* ¿Cuáles servicios de streaming están como opciones disponibles para ver series en la lista de sensacine?
* ¿Cuántas series pueden verse en cada servicio de streaming según la lista de sensacine?

In [ ]:
#TU RESPUESTA ACA

### Pregunta 2.2 (4 pts):

¿Como se distribuye la cantidad de géneros en este conjunto de series?

> **Nota**: para el conteo considere repetición de series que contengan más de un género, es decir si una serie tiene 3 generos, esta debe ser contada igualmente en cada categoria.

Para esto se le pide la siguiente tabla:

| Genero | Cantidad |
|:-----------:|:------:|
| Acción  | 38
| Comedia | 286 |
| Drama   | 462 |
| ... | ... |

Ademas, con esta información construya un gráfico de barras. Para esto pueden utilizar la librería **Matplotlib**.

> **Reminder:** Un gráfico siempre debe tener título, labels en el eje x e y, seguir los criterio de Tuffe, etc.

In [ ]:
#TU RESPUESTA

### Pregunta 2.3 (6 pts):

***
- Entregue una tabla con las 30 series con más de 2 temporadas y mayor puntaje hecho por usuarios. muestre solamente nombre, puntaje, cantidad de temporadas y cantidad de episodios

    *Ejemplo.*

| Nombre | Puntaje de usuarios | Temporadas | Episodios |
|:----------------:|:----------------:|:----------------:|:----------------:|
| Serie 1 |  Score 1 | N1|  Neps1 |
| ... | ... | ... | ... |
| Serie 30 | Score 30 | N30 | Neps30 |

***
- Muestre en una tabla el puntaje segun usuarios promedio de todos los generos (redondée a 3 decimales) y luego las estadísticas descriptivas (desviación estándar, promedio, valor máximo y mínimo) de los 3 géneros de series que tengan la mayor puntuación promedio y los 2 generos que tengan la menor puntuación promedio.

    *Ejemplo.*

| Genero   | Puntaje por usuario promedio |xx|
|:--------:|:----------------:|-|
|  Crimen | 3.878 |...|
| ...      | ...       |...|
| Drama coreano | 0.290  |...|

***
- Entregue una tabla con los servicios de streaming, la cantidad de series que se pueden observar en cada uno de ellos y el puntaje por usuario promedio de estas series, redondée a 3 decimales.

    *Ejemplo.*

| Streaming | Cantidad de series | Puntuacion por usuario promedio |
|:----------------:|:----------------:|:--------------|
|  Netflix | xx |xx|
| ... | ... |...|
| Amazon Prime | xx |xx|

***

In [ ]:
#RESPONDING

### Pregunta 2.4 (6 ptos):

*  Entregue una tabla con series que tengan una puntuacion por usuarios entre mínimo 3.5 y máximo 5.0, que tenga como género Drama, que tengan 2 o más Temporadas, que hayan terminado de emitirse y pueda verse en una plataforma de streaming.


*   ¿Cúal es la plataforma de streaming que vale la pena contratar según calidad/cantidad de series de acuerdo con los datos de Sensacine? Justifique su respuesta.

In [ ]:
#Podrias estar jugando Silksong

### Pregunta 2.5 (5 ptos):

*  Entregue una tabla con series que tengan una puntuacion por usuarios entre mínimo 4 y máximo 5.0, que tenga como género Animación, que hayan sido emitidas durante el ultimo año (2025) y pueda verse en una plataforma de streaming.


*   En base a tu analisis ¿Que serie de animación reciente recomendarias a un fan de la acción? ¿Para un fan de la comedia? Justifique sus respuestas.

In [ ]:
# Responder aquí

### Pregunta 2.6 (5 ptos):

*  Entregue una tabla con series con la serie mejor evaluada por cada año segun su fecha original de emisión.
* Presente un histograma con el puntaje promedio de las series estrenadas durante ese año por cada año.